# Ordinary Least Squares (OLS) Regression

## Complete Guide: Understanding OLS and Its Assumptions

This notebook provides a comprehensive, visual explanation of:
1. How OLS works
2. The key assumptions (Gauss-Markov conditions)
3. What happens when assumptions are violated
4. How to detect violations

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print("✓ Libraries loaded successfully!")

---
## 1. How OLS Works: Minimizing the Sum of Squared Residuals

OLS finds the line that **minimizes the sum of squared residuals** (errors).

**Goal:** Find the best β₀ (intercept) and β₁ (slope) for:
```
y = β₀ + β₁x + ε
```

Where:
- **y** = dependent variable (what we predict)
- **x** = independent variable (predictor)
- **ε** = residuals (errors)

In [ ]:
# Generate simple dataset
np.random.seed(42)
n = 50
x = np.linspace(0, 10, n)
y_true = 2 + 1.5 * x
y = y_true + np.random.normal(0, 2, n)  # Add noise

# Fit OLS model
X = x.reshape(-1, 1)
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
residuals = y - y_pred

print(f"True equation:      y = 2.0 + 1.5x")
print(f"Estimated equation: y = {model.intercept_:.2f} + {model.coef_[0]:.2f}x")
print(f"R² Score: {model.score(X, y):.4f}")

# Visualize OLS
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Data and regression line
axes[0].scatter(x, y, alpha=0.6, s=80, label='Observed data', color='steelblue')
axes[0].plot(x, y_pred, 'r-', linewidth=2, label='OLS regression line')
axes[0].plot(x, y_true, 'g--', linewidth=2, alpha=0.5, label='True relationship')
axes[0].set_xlabel('X (Independent Variable)', fontsize=12)
axes[0].set_ylabel('Y (Dependent Variable)', fontsize=12)
axes[0].set_title('OLS Regression Line', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Residuals visualization
axes[1].scatter(x, y, alpha=0.6, s=80, color='steelblue', label='Observed data')
axes[1].plot(x, y_pred, 'r-', linewidth=2, label='OLS line')

# Draw residual lines
for i in range(0, n, 3):  # Show every 3rd residual to avoid clutter
    axes[1].plot([x[i], x[i]], [y[i], y_pred[i]], 'k--', alpha=0.4, linewidth=1)

axes[1].set_xlabel('X', fontsize=12)
axes[1].set_ylabel('Y', fontsize=12)
axes[1].set_title('Residuals: Distance from Line', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ OLS minimizes the sum of squared residuals (vertical distances)!")

---
## 2. The OLS Assumptions (Gauss-Markov Conditions)

For OLS to provide the **Best Linear Unbiased Estimator (BLUE)**, the following assumptions must hold:

### Core Assumptions:
1. **Linearity**: The relationship is linear in parameters
2. **Zero Mean of Residuals**: E(ε) = 0
3. **No Correlation between X and Residuals**: Cov(X, ε) = 0
4. **Homoscedasticity**: Constant variance of residuals (Var(ε) = σ²)
5. **No Autocorrelation**: Residuals are uncorrelated (Cov(εᵢ, εⱼ) = 0 for i ≠ j)
6. **Normality of Residuals**: ε ~ N(0, σ²) (needed for inference)
7. **No Multicollinearity**: Independent variables are uncorrelated (for multiple regression)

Let's examine each assumption!

---
## 3. Assumption 1: Linearity

**Requirement:** The relationship between X and Y must be linear (in parameters).

**What happens if violated?** Predictions will be biased and inaccurate.

In [ ]:
# Create non-linear relationship
x_nl = np.linspace(0, 10, 100)
y_nl_true = 2 + 3*x_nl - 0.3*x_nl**2  # Quadratic relationship
y_nl = y_nl_true + np.random.normal(0, 2, 100)

# Fit linear model (wrong!)
X_nl = x_nl.reshape(-1, 1)
model_linear = LinearRegression().fit(X_nl, y_nl)
y_nl_pred_linear = model_linear.predict(X_nl)
residuals_nl = y_nl - y_nl_pred_linear

# Fit correct quadratic model
X_nl_poly = np.column_stack([x_nl, x_nl**2])
model_poly = LinearRegression().fit(X_nl_poly, y_nl)
y_nl_pred_poly = model_poly.predict(X_nl_poly)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Plot 1: Wrong linear fit
axes[0].scatter(x_nl, y_nl, alpha=0.5, s=30, label='Data')
axes[0].plot(x_nl, y_nl_pred_linear, 'r-', linewidth=2, label='Linear fit (WRONG)')
axes[0].plot(x_nl, y_nl_true, 'g--', linewidth=2, alpha=0.7, label='True quadratic')
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title('❌ Linearity Violated', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Residual plot shows pattern (BAD!)
axes[1].scatter(y_nl_pred_linear, residuals_nl, alpha=0.5, s=30)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Fitted Values')
axes[1].set_ylabel('Residuals')
axes[1].set_title('❌ Residual Pattern → Non-linearity', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

# Plot 3: Correct polynomial fit
axes[2].scatter(x_nl, y_nl, alpha=0.5, s=30, label='Data')
axes[2].plot(x_nl, y_nl_pred_poly, 'r-', linewidth=2, label='Quadratic fit (CORRECT)')
axes[2].plot(x_nl, y_nl_true, 'g--', linewidth=2, alpha=0.7, label='True quadratic')
axes[2].set_xlabel('X')
axes[2].set_ylabel('Y')
axes[2].set_title('✓ Correct Model', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Detection: Look for patterns in residual plots")
print("🛠️  Solution: Transform variables or use polynomial regression")

---
## 4. Assumption 2: Zero Mean of Residuals

**Requirement:** E(ε) = 0 → Residuals should average to zero.

**Good news:** OLS automatically ensures this by including an intercept!

In [ ]:
# Check mean of residuals from our earlier model
print(f"Mean of residuals: {np.mean(residuals):.10f}")
print(f"(Essentially zero, due to floating-point precision)\n")

# Visualize distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(residuals, bins=20, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].axvline(x=np.mean(residuals), color='r', linestyle='--', linewidth=2, 
                label=f'Mean = {np.mean(residuals):.4f}')
axes[0].set_xlabel('Residuals', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Residuals', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(residuals, vert=True)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2, label='Zero line')
axes[1].set_ylabel('Residuals', fontsize=12)
axes[1].set_title('Residual Box Plot', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ When model includes intercept, mean of residuals = 0 (by construction)")

---
## 5. Assumption 3: No Correlation Between X and Residuals

**Requirement:** Cov(X, ε) = 0 → Residuals should be independent of X.

**What happens if violated?** Estimates are biased (omitted variable bias).

In [ ]:
# Check correlation
correlation = np.corrcoef(x, residuals)[0, 1]
print(f"Correlation(X, residuals) = {correlation:.10f}")
print("(Should be approximately zero)\n")

# Visualize
plt.figure(figsize=(10, 5))
plt.scatter(x, residuals, alpha=0.6, s=60, color='steelblue')
plt.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Zero line')
plt.xlabel('X (Independent Variable)', fontsize=12)
plt.ylabel('Residuals', fontsize=12)
plt.title(f'Residuals vs X (Correlation = {correlation:.4f})', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("✓ No pattern → Assumption satisfied")
print("❌ Pattern or trend → Omitted variable problem!")

---
## 6. Assumption 4: Homoscedasticity (Constant Variance)

**Requirement:** Var(ε) = σ² → Variance of residuals should be constant across all X values.

**What happens if violated (Heteroscedasticity)?**
- Standard errors are incorrect
- Confidence intervals and hypothesis tests are invalid
- Estimates are still unbiased but not efficient

In [ ]:
# Create heteroscedastic data (variance increases with X)
x_het = np.linspace(1, 10, 100)
y_het = 2 + 1.5 * x_het + np.random.normal(0, 1, 100) * x_het  # Variance grows with x!

# Fit model
X_het = x_het.reshape(-1, 1)
model_het = LinearRegression().fit(X_het, y_het)
y_het_pred = model_het.predict(X_het)
residuals_het = y_het - y_het_pred

# Compare with homoscedastic data
y_homo = 2 + 1.5 * x_het + np.random.normal(0, 2, 100)  # Constant variance
model_homo = LinearRegression().fit(X_het, y_homo)
y_homo_pred = model_homo.predict(X_het)
residuals_homo = y_homo - y_homo_pred

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Homoscedastic case
axes[0, 0].scatter(x_het, y_homo, alpha=0.5, s=30, color='green')
axes[0, 0].plot(x_het, y_homo_pred, 'r-', linewidth=2)
axes[0, 0].set_xlabel('X')
axes[0, 0].set_ylabel('Y')
axes[0, 0].set_title('✓ Homoscedastic Data', fontsize=12, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

axes[0, 1].scatter(y_homo_pred, residuals_homo, alpha=0.5, s=30, color='green')
axes[0, 1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Fitted Values')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('✓ Constant Variance', fontsize=12, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Heteroscedastic case
axes[1, 0].scatter(x_het, y_het, alpha=0.5, s=30, color='red')
axes[1, 0].plot(x_het, y_het_pred, 'r-', linewidth=2)
axes[1, 0].set_xlabel('X')
axes[1, 0].set_ylabel('Y')
axes[1, 0].set_title('❌ Heteroscedastic Data', fontsize=12, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

axes[1, 1].scatter(y_het_pred, residuals_het, alpha=0.5, s=30, color='red')
axes[1, 1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Fitted Values')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('❌ Variance Increases (Funnel Shape)', fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Detection: Look for funnel/cone shape in residual plot")
print("🛠️  Solutions:")
print("   • Transform Y (e.g., log transform)")
print("   • Use Weighted Least Squares (WLS)")
print("   • Use robust standard errors")

---
## 7. Assumption 5: No Autocorrelation

**Requirement:** Residuals should be uncorrelated → Cov(εᵢ, εⱼ) = 0 for i ≠ j.

**What happens if violated?**
- Common in time series data
- Standard errors are underestimated
- Over-confident predictions

In [ ]:
# Create autocorrelated residuals (time series)
n_time = 100
t = np.arange(n_time)

# Data WITH autocorrelation
np.random.seed(42)
errors_auto = np.zeros(n_time)
errors_auto[0] = np.random.normal(0, 1)
for i in range(1, n_time):
    errors_auto[i] = 0.8 * errors_auto[i-1] + np.random.normal(0, 1)  # AR(1) process

y_auto = 2 + 0.5 * t + errors_auto

# Data WITHOUT autocorrelation
errors_no_auto = np.random.normal(0, 2, n_time)
y_no_auto = 2 + 0.5 * t + errors_no_auto

# Fit models
T = t.reshape(-1, 1)
model_auto = LinearRegression().fit(T, y_auto)
residuals_auto = y_auto - model_auto.predict(T)

model_no_auto = LinearRegression().fit(T, y_no_auto)
residuals_no_auto = y_no_auto - model_no_auto.predict(T)

# Calculate autocorrelation (lag 1)
auto_corr = np.corrcoef(residuals_auto[:-1], residuals_auto[1:])[0, 1]
no_auto_corr = np.corrcoef(residuals_no_auto[:-1], residuals_no_auto[1:])[0, 1]

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# No autocorrelation
axes[0, 0].plot(t, residuals_no_auto, marker='o', alpha=0.6, linestyle='-', markersize=4)
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('✓ No Autocorrelation', fontsize=12, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

axes[0, 1].scatter(residuals_no_auto[:-1], residuals_no_auto[1:], alpha=0.5, s=30, color='green')
axes[0, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 1].set_xlabel('Residual at t')
axes[0, 1].set_ylabel('Residual at t+1')
axes[0, 1].set_title(f'✓ Lag Plot (r = {no_auto_corr:.3f})', fontsize=12, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# WITH autocorrelation
axes[1, 0].plot(t, residuals_auto, marker='o', alpha=0.6, linestyle='-', 
                markersize=4, color='red')
axes[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Time')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].set_title('❌ Autocorrelation Present', fontsize=12, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

axes[1, 1].scatter(residuals_auto[:-1], residuals_auto[1:], alpha=0.5, s=30, color='red')
axes[1, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1, 1].axvline(x=0, color='gray', linestyle='--', alpha=0.5)
# Add trend line
z = np.polyfit(residuals_auto[:-1], residuals_auto[1:], 1)
p = np.poly1d(z)
axes[1, 1].plot(residuals_auto[:-1], p(residuals_auto[:-1]), "b--", linewidth=2, alpha=0.7)
axes[1, 1].set_xlabel('Residual at t')
axes[1, 1].set_ylabel('Residual at t+1')
axes[1, 1].set_title(f'❌ Lag Plot (r = {auto_corr:.3f})', fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Detection: Durbin-Watson test, ACF/PACF plots, or lag plots")
print("🛠️  Solutions:")
print("   • Add lagged variables")
print("   • Use time series models (ARIMA, etc.)")
print("   • Use Newey-West standard errors")

---
## 8. Assumption 6: Normality of Residuals

**Requirement:** ε ~ N(0, σ²) → Residuals should be normally distributed.

**What happens if violated?**
- Estimates remain unbiased
- Hypothesis tests and confidence intervals may be invalid
- Less critical with large samples (Central Limit Theorem)

In [ ]:
# Create data with non-normal residuals
x_norm = np.linspace(0, 10, 100)

# Normal residuals
errors_normal = np.random.normal(0, 2, 100)
y_normal = 2 + 1.5 * x_norm + errors_normal

# Non-normal residuals (skewed)
errors_skewed = np.random.exponential(2, 100) - 2  # Right-skewed
y_skewed = 2 + 1.5 * x_norm + errors_skewed

# Fit models
X_norm = x_norm.reshape(-1, 1)
model_normal = LinearRegression().fit(X_norm, y_normal)
residuals_normal = y_normal - model_normal.predict(X_norm)

model_skewed = LinearRegression().fit(X_norm, y_skewed)
residuals_skewed = y_skewed - model_skewed.predict(X_norm)

# Statistical tests
from scipy.stats import shapiro, probplot

stat_normal, p_normal = shapiro(residuals_normal)
stat_skewed, p_skewed = shapiro(residuals_skewed)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Normal residuals
axes[0, 0].hist(residuals_normal, bins=20, alpha=0.7, color='green', edgecolor='black')
axes[0, 0].set_xlabel('Residuals')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title(f'✓ Normal Residuals\nShapiro p={p_normal:.4f}', 
                     fontsize=11, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

probplot(residuals_normal, dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('✓ Q-Q Plot: Points on Line', fontsize=11, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

axes[0, 2].boxplot(residuals_normal)
axes[0, 2].set_ylabel('Residuals')
axes[0, 2].set_title('✓ Symmetric Distribution', fontsize=11, fontweight='bold')
axes[0, 2].grid(alpha=0.3)

# Skewed residuals
axes[1, 0].hist(residuals_skewed, bins=20, alpha=0.7, color='red', edgecolor='black')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title(f'❌ Skewed Residuals\nShapiro p={p_skewed:.4f}', 
                     fontsize=11, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

probplot(residuals_skewed, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('❌ Q-Q Plot: Deviation from Line', fontsize=11, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

axes[1, 2].boxplot(residuals_skewed)
axes[1, 2].set_ylabel('Residuals')
axes[1, 2].set_title('❌ Skewed with Outliers', fontsize=11, fontweight='bold')
axes[1, 2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Detection: Shapiro-Wilk test, Q-Q plot, histogram")
print("   • Shapiro-Wilk p > 0.05 → Likely normal")
print("   • Q-Q plot points on line → Normal")
print("\n🛠️  Solutions:")
print("   • Transform Y (Box-Cox, log, sqrt)")
print("   • Remove outliers (carefully!)")
print("   • Use robust regression methods")

---
## 9. Assumption 7: No Multicollinearity (Multiple Regression)

**Requirement:** Independent variables should not be highly correlated with each other.

**What happens if violated?**
- Unstable coefficient estimates
- Large standard errors
- Difficult to determine individual variable effects

In [ ]:
# Create multicollinear data
np.random.seed(42)
n = 100

# Independent variables
X1 = np.random.randn(n)
X2 = np.random.randn(n)
X3 = 0.9 * X1 + 0.1 * np.random.randn(n)  # Highly correlated with X1!

# True relationship: y = 2 + 3*X1 + 4*X2 + noise
y_multi = 2 + 3*X1 + 4*X2 + np.random.randn(n)

# Case 1: No multicollinearity (X1 and X2)
X_good = np.column_stack([X1, X2])
model_good = LinearRegression().fit(X_good, y_multi)

# Case 2: WITH multicollinearity (X1 and X3)
X_bad = np.column_stack([X1, X3])
model_bad = LinearRegression().fit(X_bad, y_multi)

print("🔍 Multicollinearity Check:\n")
print(f"Correlation(X1, X2): {np.corrcoef(X1, X2)[0,1]:.4f} ✓ (Low)")
print(f"Correlation(X1, X3): {np.corrcoef(X1, X3)[0,1]:.4f} ❌ (High!)\n")

print("Model WITHOUT multicollinearity (X1, X2):")
print(f"  β₁ = {model_good.coef_[0]:.3f} (should be ~3)")
print(f"  β₂ = {model_good.coef_[1]:.3f} (should be ~4)")
print(f"  R² = {model_good.score(X_good, y_multi):.4f}\n")

print("Model WITH multicollinearity (X1, X3):")
print(f"  β₁ = {model_bad.coef_[0]:.3f} (unstable!)")
print(f"  β₃ = {model_bad.coef_[1]:.3f} (unstable!)")
print(f"  R² = {model_bad.score(X_bad, y_multi):.4f}")
print("  → Coefficients are unreliable!\n")

# Visualize correlation
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Good: X1 vs X2
axes[0].scatter(X1, X2, alpha=0.6, s=40, color='green')
axes[0].set_xlabel('X1')
axes[0].set_ylabel('X2')
axes[0].set_title(f'✓ No Multicollinearity\nr = {np.corrcoef(X1, X2)[0,1]:.3f}', 
                  fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)

# Bad: X1 vs X3
axes[1].scatter(X1, X3, alpha=0.6, s=40, color='red')
# Add trend line
z = np.polyfit(X1, X3, 1)
p = np.poly1d(z)
axes[1].plot(np.sort(X1), p(np.sort(X1)), "b--", linewidth=2)
axes[1].set_xlabel('X1')
axes[1].set_ylabel('X3')
axes[1].set_title(f'❌ High Multicollinearity\nr = {np.corrcoef(X1, X3)[0,1]:.3f}', 
                  fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

# Correlation matrix
corr_matrix = np.corrcoef([X1, X2, X3])
im = axes[2].imshow(corr_matrix, cmap='RdYlGn_r', vmin=-1, vmax=1)
axes[2].set_xticks([0, 1, 2])
axes[2].set_yticks([0, 1, 2])
axes[2].set_xticklabels(['X1', 'X2', 'X3'])
axes[2].set_yticklabels(['X1', 'X2', 'X3'])
axes[2].set_title('Correlation Matrix', fontsize=12, fontweight='bold')

# Add correlation values
for i in range(3):
    for j in range(3):
        text = axes[2].text(j, i, f'{corr_matrix[i, j]:.2f}',
                           ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im, ax=axes[2])
plt.tight_layout()
plt.show()

print("🔍 Detection: Variance Inflation Factor (VIF), correlation matrix")
print("   • VIF > 10 → Severe multicollinearity")
print("   • |correlation| > 0.8 → Potential problem")
print("\n🛠️  Solutions:")
print("   • Remove one of the correlated variables")
print("   • Combine correlated variables (PCA)")
print("   • Use Ridge regression")

---
## 10. Complete Diagnostic Plot

A comprehensive residual analysis to check all assumptions at once.

In [ ]:
# Use our original good model
from scipy import stats as sp_stats

# Standardized residuals
residuals_std = (residuals - np.mean(residuals)) / np.std(residuals)

# Create comprehensive diagnostic plot
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Residuals vs Fitted
ax1 = fig.add_subplot(gs[0, :])
ax1.scatter(y_pred, residuals, alpha=0.6, s=50)
ax1.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax1.set_xlabel('Fitted Values', fontsize=11)
ax1.set_ylabel('Residuals', fontsize=11)
ax1.set_title('1. Residuals vs Fitted (Check: Linearity, Homoscedasticity)', 
              fontsize=12, fontweight='bold')
ax1.grid(alpha=0.3)

# 2. Q-Q Plot
ax2 = fig.add_subplot(gs[1, 0])
sp_stats.probplot(residuals, dist="norm", plot=ax2)
ax2.set_title('2. Q-Q Plot\n(Check: Normality)', fontsize=11, fontweight='bold')
ax2.grid(alpha=0.3)

# 3. Scale-Location (sqrt of standardized residuals)
ax3 = fig.add_subplot(gs[1, 1])
ax3.scatter(y_pred, np.sqrt(np.abs(residuals_std)), alpha=0.6, s=50)
ax3.set_xlabel('Fitted Values', fontsize=11)
ax3.set_ylabel('√|Standardized Residuals|', fontsize=11)
ax3.set_title('3. Scale-Location\n(Check: Homoscedasticity)', fontsize=11, fontweight='bold')
ax3.grid(alpha=0.3)

# 4. Residuals histogram
ax4 = fig.add_subplot(gs[1, 2])
ax4.hist(residuals, bins=20, alpha=0.7, color='steelblue', edgecolor='black')
ax4.axvline(x=0, color='r', linestyle='--', linewidth=2)
ax4.set_xlabel('Residuals', fontsize=11)
ax4.set_ylabel('Frequency', fontsize=11)
ax4.set_title('4. Histogram\n(Check: Normality)', fontsize=11, fontweight='bold')
ax4.grid(alpha=0.3)

# 5. Residuals vs X
ax5 = fig.add_subplot(gs[2, 0])
ax5.scatter(x, residuals, alpha=0.6, s=50, color='purple')
ax5.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax5.set_xlabel('X', fontsize=11)
ax5.set_ylabel('Residuals', fontsize=11)
ax5.set_title('5. Residuals vs X\n(Check: Independence)', fontsize=11, fontweight='bold')
ax5.grid(alpha=0.3)

# 6. Residuals vs Order
ax6 = fig.add_subplot(gs[2, 1])
ax6.plot(range(len(residuals)), residuals, marker='o', linestyle='-', 
         alpha=0.6, markersize=4, color='orange')
ax6.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax6.set_xlabel('Observation Order', fontsize=11)
ax6.set_ylabel('Residuals', fontsize=11)
ax6.set_title('6. Residuals vs Order\n(Check: Autocorrelation)', fontsize=11, fontweight='bold')
ax6.grid(alpha=0.3)

# 7. Box plot
ax7 = fig.add_subplot(gs[2, 2])
ax7.boxplot(residuals, vert=True)
ax7.axhline(y=0, color='r', linestyle='--', linewidth=2)
ax7.set_ylabel('Residuals', fontsize=11)
ax7.set_title('7. Box Plot\n(Check: Outliers)', fontsize=11, fontweight='bold')
ax7.grid(alpha=0.3)

plt.suptitle('OLS Regression Diagnostics', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("\n✓ Use these plots together to assess all OLS assumptions!")

---
## 11. Summary: OLS Assumptions Checklist

### ✅ Quick Checklist

| Assumption | What to Check | What You Want to See |
|-----------|---------------|---------------------|
| **1. Linearity** | Residuals vs Fitted plot | No pattern, random scatter |
| **2. Zero Mean** | Mean of residuals | ≈ 0 (automatic with intercept) |
| **3. No X-ε Correlation** | Residuals vs X plot | No pattern, correlation ≈ 0 |
| **4. Homoscedasticity** | Residuals vs Fitted plot | Constant spread (no funnel) |
| **5. No Autocorrelation** | Residuals vs Order plot | No pattern over time |
| **6. Normality** | Q-Q plot, Shapiro test | Points on line, p > 0.05 |
| **7. No Multicollinearity** | Correlation matrix, VIF | Low correlations, VIF < 10 |

### 🎯 What to Do When Assumptions Fail

**Non-linearity**
- Transform variables (log, polynomial, etc.)
- Use non-linear models

**Heteroscedasticity**
- Transform Y (log, Box-Cox)
- Use Weighted Least Squares (WLS)
- Use robust standard errors

**Autocorrelation**
- Add time lags
- Use time series models (ARIMA)
- Use Newey-West standard errors

**Non-normality**
- Transform Y
- Use robust regression
- Remove outliers (carefully!)
- Remember: Less critical with large samples

**Multicollinearity**
- Remove correlated variables
- Use PCA
- Use Ridge/Lasso regression

### 📊 Key Takeaways

1. **OLS is BLUE** (Best Linear Unbiased Estimator) **only when assumptions hold**
2. **Always check residual plots** - they reveal assumption violations
3. **Some violations are worse than others:**
   - Omitted variables → Biased estimates (serious!)
   - Heteroscedasticity → Invalid inference (fixable)
   - Non-normality → Less critical with large n
4. **Perfect assumptions are rare** - focus on major violations
5. **Multiple checks are better** - no single test is definitive

In [ ]:
# Final visualization: Assumption violations summary
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

violation_types = [
    'Good Model',
    'Non-linearity',
    'Heteroscedasticity',
    'Autocorrelation',
    'Non-normality',
    'Outliers'
]

# Generate example patterns for each
np.random.seed(42)
x_demo = np.linspace(0, 10, 100)

patterns = [
    np.random.normal(0, 1, 100),  # Good
    (x_demo - 5)**2 - 5,  # Non-linear
    np.random.normal(0, 1, 100) * (1 + 0.3*x_demo),  # Heteroscedastic
    np.cumsum(np.random.normal(0, 0.3, 100)),  # Autocorrelated
    np.concatenate([np.random.normal(-1, 0.5, 70), np.random.normal(3, 0.5, 30)]),  # Skewed
    np.concatenate([np.random.normal(0, 1, 95), [8, -7, 9, -8, 7]])  # With outliers
]

colors = ['green', 'red', 'red', 'red', 'red', 'red']
symbols = ['✓', '❌', '❌', '❌', '❌', '❌']

for idx, (pattern, title, color, symbol) in enumerate(zip(patterns, violation_types, colors, symbols)):
    if idx in [0, 1, 2, 5]:  # Scatter plots
        axes[idx].scatter(x_demo[:len(pattern)], pattern, alpha=0.5, s=30, color=color)
    else:  # Line plots for time-based
        axes[idx].plot(range(len(pattern)), pattern, alpha=0.6, color=color)
    
    axes[idx].axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    axes[idx].set_title(f'{symbol} {title}', fontsize=12, fontweight='bold')
    axes[idx].grid(alpha=0.3)

plt.suptitle('Visual Guide to OLS Assumption Violations', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("REMEMBER: Check your assumptions, check your residuals!")
print("="*70)